In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ukwud\\OneDrive - Aston University\\Documents\\GitHub\\text-summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
print(os.getcwd())

c:\Users\ukwud\OneDrive - Aston University\Documents\GitHub\text-summarizer


In [5]:
#1. config.yaml
#2. params.yaml
#3. entity
#4. utils
#5. logging
#6. data ingestion
#7. data validation
#8. data transformation
#9. model trainer

In [6]:
#entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: 3
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    logging_steps: int
    gradiant_accummulation_steps: int
    save_steps: int
    weight_decay: float
    eval_steps: int
    evaluation_strategy: str

In [7]:
#utilities
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories
from text_summarizer.logging import logger
from datasets import load_dataset, load_from_disk

c:\Users\ukwud\OneDrive - Aston University\Documents\GitHub\text-summarizer\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-02-16 23:04:43,660: INFO: config: PyTorch version 2.5.1 available.]


In [15]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path= config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps= params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            per_device_eval_batch_size= params.per_device_train_batch_size,
            logging_steps= params.logging_steps,
            gradiant_accummulation_steps= params.gradiant_accummulation_steps,
            save_steps= params.save_steps,
            weight_decay= params.weight_decay,
            eval_steps= params.eval_steps,
            evaluation_strategy= params.evaluation_strategy
        )

        return model_trainer_config


In [16]:
#components
from text_summarizer.logging import logger
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import Trainer, TrainingArguments
import torch

In [17]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            gradient_accumulation_steps=self.config.gradiant_accummulation_steps,
            save_steps=self.config.save_steps
        )

        trainer = Trainer(model = model_pegasus, args = trainer_args,
                          tokenizer = tokenizer, data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["train"],
                          eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Saving the Model
        model_pegasus.save_pretrtained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        ## Saving the tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

        

In [18]:
#pipeline

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-02-16 23:34:21,185: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-16 23:34:21,197: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-16 23:34:21,199: INFO: common: directory created at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\ukwud\OneDrive - Aston University\Documents\GitHub\text-summarizer\venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ukwud\AppData\Local\Temp\ipykernel_20432\4194866270.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model = model_pegasus, args = trainer_args,


Step,Training Loss,Validation Loss


c:\Users\ukwud\OneDrive - Aston University\Documents\GitHub\text-summarizer\venv\Lib\site-packages\transformers\modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


KeyboardInterrupt: 